In [1]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import os

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df_train = pd.read_csv('./Output/csv/train_engineer.csv')
df_test = pd.read_csv('./Output/csv/test_engineer.csv')

In [3]:
target = df_train['SalePrice']
df_train.drop(['SalePrice'], axis = 1, inplace = True)

In [5]:
overfit = []
for i in df_train.columns:
    counts = df_train[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(df_train) * 100 > 99.94:
        overfit.append(i)
df_train = df_train.drop(overfit, axis=1)
df_test = df_test.drop(overfit, axis=1)

In [6]:
# kfold is 10
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
# make the function to get the score
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
def cv_rmse(model, X=df_train):
    rmse = np.sqrt(-cross_val_score(model, X, target, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [7]:
# setup models
# all the parameter is based on existed data processes
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=kfolds, l1_ratio=e_l1ratio))
                                        
svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
                                       

xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

# stack
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                            gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [8]:
# ridge
# linear regreesion without feature importance
score = cv_rmse(ridge)
score.mean()

0.12086470974541093

In [9]:
# lasso
# linear regreesion with no feature importance
score = cv_rmse(lasso)
score.mean()

0.11846931843258668

In [10]:
# elastic net
# linear model with no feature importance
score = cv_rmse(elasticnet)
score.mean()

0.11864749467553612

In [11]:
# svr
score = cv_rmse(svr)
score.mean()

0.12068343273696454

In [12]:
from sklearn.ensemble import RandomForestRegressor
# random Forest Regression
forest_reg = RandomForestRegressor(n_estimators=40, random_state=42, 
                                   min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
score = cv_rmse(forest_reg)
score.mean()

0.1366595518206411

In [13]:
# lightgbm
score = cv_rmse(lightgbm)
score.mean()

0.12087977828691798

In [14]:
# gbr
score = cv_rmse(gbr)
score.mean()

0.12042945359860377

In [15]:
# xgboost
score = cv_rmse(xgboost)
score.mean()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:42:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:43:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:48:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:49:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.11833366806900762

In [17]:
# Plot feature importance
def plot_feature_importance(model, df):
    feature_importance = model.feature_importances_[:30]
    # make importances relative to max importance
    plt.figure(figsize=(20, 20)) #figure size
    #making it a percentage relative to the max value
    feature_importance = 100.0 * (feature_importance / feature_importance.max()) 
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    #used train_drop here to show the name of each feature instead of our train_prepared
    plt.yticks(pos, df.columns[sorted_idx], fontsize=15) 
    plt.xlabel('Relative Importance', fontsize=20)
    plt.ylabel('Features', fontsize=20)
    plt.title('Variable Importance', fontsize=30)
    return pd.DataFrame({'columns_name': df.columns, 'feature_importance': model.feature_importances_})

In [31]:
stack_gen_model = stack_gen.fit(np.array(df_train), np.array(target))
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(stack_gen_model.predict(np.array(df_test))))
submission.to_csv('./output/csv/stack_gen.csv', index = False)
# 0.12029

In [32]:
elastic_model = elasticnet.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(elastic_model.predict(df_test)))
submission.to_csv('./output/csv/elastic.csv', index = False)
# 0.11941

In [33]:
lasso_model = lasso.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(lasso_model.predict(df_test)))
submission.to_csv('./output/csv/lasso.csv', index = False)
# 0.11941

In [34]:
ridge_model = ridge.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(ridge_model.predict(df_test)))
submission.to_csv('./output/csv/ridge.csv', index = False)
# 0.11926

In [35]:
svr_model = svr.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(svr_model.predict(df_test)))
submission.to_csv('./output/csv/svr.csv', index = False)
# 0.12029

In [36]:
forest_reg_model = forest_reg.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(forest_reg.predict(df_test)))
submission.to_csv('./output/csv/forest.csv', index = False)
# 0.14152

In [37]:
lightgbm_model = lightgbm.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(lightgbm.predict(df_test)))
submission.to_csv('./output/csv/lightgbm.csv', index = False)
# 0.12105

In [38]:
gbr_model = gbr.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(gbr_model.predict(df_test)))
submission.to_csv('./output/csv/gbr.csv', index = False)
# 0.12205

In [39]:
xgboost_model = xgboost.fit(df_train, target)
submission = pd.read_csv("./Resources/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(xgboost_model.predict(df_test)))
submission.to_csv('./output/csv/xgboost.csv', index = False)
# 0.12572